In [56]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

from pathlib import Path
import numpy as np
# 当前脚本所在目录
#current_dir = Path(__file__).parent
#word2vec_output_file = 'glove.6B.300d.txt.word2vec'
word2vec_output_file = 'glove.42B.300d.txt.word2vec'
# 加载转换后的文件
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

#keywords1 = ["price", "City", "Country", "Start","Date","EndDate","Package","Flights"," Hotel"]
threshold = 0.5 
userInput = "what's the price to Tokyo"
#userInput = "what's the weather?"

# 定义计算平均向量的函数
def average_vector(words):
    # 过滤掉模型词汇表中不存在的词
    valid_words = [word for word in words if word in model]
    if not valid_words:
        return None
    # 计算平均向量
    return np.mean([model[word] for word in valid_words], axis=0)

# 函数：计算余弦相似度
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [72]:
#keywords1 = ["white","price", "City","Country", "Start","Date","End","Package","Type","Flights","Hotel"]
keywords1 = ['Comment', 'Destinations', 'Package', 'Review', 'Customer', 'ID', 'Rating', 'Price', 'Description', 'End', 'City', 'And', 'Travel', 'Attractions', 'Destination', 'Packages', 'Feedback', 'Type', 'Date', 'Country', 'Reviews', 'Start', 'Includes']
keywords1 = [element.lower() for element in keywords1]
#userInput = "what's the price to Tokyo"

userInput = "describe"
#userInput = "what's the weather"
#userVec = average_vector(userInput)
userVec = model[userInput]
len(userVec)
#keyVec = average_vector(keywords1)

300

In [73]:

not_included = [word for word in keywords1 if word not in model.key_to_index]
similarities = [cosine_similarity(userVec, model[keyword]) for keyword in keywords1 if keyword in model]
print(not_included)
print(similarities)
#print(cosine_similarity(userVec, keyVec))



[]
[0.41214254, 0.24890724, 0.32484734, 0.41178823, 0.41867498, 0.3018977, 0.26095286, 0.2960444, 0.53462183, 0.44081, 0.3397033, 0.46377265, 0.30374947, 0.22094099, 0.33016422, 0.26242846, 0.34614113, 0.5006737, 0.31471473, 0.41468817, 0.33375677, 0.46703905, 0.40216398]


In [84]:
import logging
from colorlog import ColoredFormatter

# 创建一个日志记录器
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)

# # 创建一个流处理器
# stream_handler = logging.StreamHandler()

# # 设置颜色格式
# formatter = ColoredFormatter(
#     "%(log_color)s%(levelname)-8s%(reset)s %(blue)s%(message)s",
#     datefmt=None,
#     reset=True,
#     log_colors={
#         'DEBUG': 'cyan',
#         'INFO': 'green',
#         'WARNING': 'yellow',
#         'ERROR': 'red',
#         'CRITICAL': 'red,bg_white',
#     },
#     secondary_log_colors={},
#     style='%'
# )

# #stream_handler.setFormatter(formatter)
# logger.addHandler(stream_handler)

# # 测试不同级别的日志输出
logger.debug('一个调试消息')
# logger.info('一个信息消息')
# logger.warning('一个警告消息')
# logger.error('一个错误消息')
# logger.critical('一个严重错误消息')


DEBUG    一个调试消息
一个调试消息
DEBUG    一个调试消息
一个调试消息
一个调试消息
DEBUG:my_logger:一个调试消息


In [107]:
def trimquery(sqlquery):
    for tag in ["```sql", "```SQL", "```mysql", "```MYSQL"]:
        start = sqlquery.find(tag)
        if start != -1:
            # 标记后面跟着的是SQL语句，所以要跳过标记本身
            start += len(tag)
            break  # 找到一个就停止搜索
        
    if start == -1:
        # 如果没有找到任何标记
        logger.warning("SQL query start tag not found.")
        return "SQL query start tag not found."
    
    startqry = sqlquery[start:]
    #print(startqry)
    end = startqry.find("```")
    if end == -1:
        # 如果找不到结束标记，可以返回整个提取的字符串或处理错误
        logger.warning("SQL query end tag not found.")
        return "SQL query end tag not found."
    
    endquery = startqry[0:end] 
    
    return endquery

# 示例文本块
text_block = """
Based on your table schemas, you can get the price to Tokyo with the following SQL query:

```SQL
SELECT Price
FROM TravelPackages
WHERE Destination = "Tokyo";
```
This query selects the price from the TravelPackages table where the Destination field is equal to "Tokyo". Be sure to replace `Tokyo` with the exact city name as it appears in your table data if it's not exactly "Tokyo".

Please note that this will return the prices of all listed packages for Tokyo. If you need to narrow the results (ex. by a specific PackageType or date range), you will need to add additional conditions to the WHERE clause.
"""


In [108]:
extracted_query = trimquery(text_block)
print(extracted_query)


SELECT Price
FROM TravelPackages
WHERE Destination = "Tokyo";

